# Label-Agnostic Bayesian Optimization for IDS (Label Trainer)
## 1. Introduction

This notebook tunes hyperparameters for an Intrusion Detection System (IDS) label trainer using **Bayesian Optimisation (BO)** and compares the result to a compact **1D-CNN** baseline. We optimise **PR-AUC** (Average Precision) via stratified cross-validation and then select a decision threshold $\tau$ on the test set for **max-F1** or a **target Precision**.  

Symbols and terms align with the Imperial **Professional Certificate in Machine Learning and AI** modules:
- **Module 3:** Probabilistic modelling (Gaussian pdf/cdf), evaluation under imbalance (Precision–Recall curves).
- **Module 10:** Model selection & black-box optimisation (hyperparameter tuning workflows, ensembles).
- **Bandits/RL link:** Explore–exploit intuition for BO acquisition functions (see multi-armed bandit formulation).

---

### Why Hyperparameter Optimisation Matters
We seek hyperparameters $\theta$ (e.g., `learning_rate`, `num_leaves`, `max_depth`) that maximise a validation objective:
\[
J(\theta) = \text{PR-AUC from stratified CV}.
\]

In IDS:
- Training is **expensive** (multiple folds).
- Scores are **noisy** (variance across folds).
- The function is **non-convex** (many local optima).  

- **Grid search** is exhaustive but wasteful.  
- **Random search** ignores prior trials.  
- **Bayesian Optimisation** uses a surrogate + acquisition rule to prioritise *informative* trials, reaching good models in fewer iterations.

---

### Bayesian Optimisation: Concept
Treat $J(\theta)$ as a black-box function $f(\theta)$. BO repeats:
1. Evaluate initial $\theta_i$, record $y_i=J(\theta_i)$.
2. Fit a **surrogate model** (often Gaussian Process) giving posterior mean $\mu_t(\theta)$ and uncertainty $\sigma_t(\theta)$.
3. Choose next $\theta$ by maximising an **acquisition function** $a_t(\theta)$ that balances **exploration** (high $\sigma_t$) and **exploitation** (high $\mu_t$).
4. Repeat until budget exhausted; return best $\theta$.

---

### Surrogate Model (Gaussian Process, Module 3)
Given $\mathcal D_t=\{(\theta_i,y_i)\}_{i=1}^t$ with $y_i=J(\theta_i)+\epsilon_i$, the GP posterior is:
\[
$\mu_t(\theta) = k(\theta,\Theta_t)K_t^{-1}\mathbf y_t, \quad$ 
$\sigma_t^2(\theta) = k(\theta,\theta) - k(\theta,\Theta_t)K_t^{-1}k(\Theta_t,\theta)$.
\]

---

### Acquisition Functions (Module 10, Bandits Explore–Exploit)
Let $f^*=\max_{i\le t} y_i$. Using Gaussian $\Phi,\phi$:

- **Expected Improvement (EI):**
\[
$EI(\theta) = (\mu_t-f^*-\xi)\,\Phi(z) + \sigma_t\,\phi(z),\quad z=\frac{\mu_t-f^*-\xi}{\sigma_t}$
\]
- **Probability of Improvement (PI):**
\[
$PI(\theta)=\Phi\!\left(\frac{\mu_t-f^*-\xi}{\sigma_t}\right)$
\]
- **Upper Confidence Bound (UCB):**
\[
$UCB(\theta)=\mu_t + \kappa\,\sigma_t$
\]

Parameters $\xi\ge 0$ and $\kappa>0$ control exploration vs exploitation — exactly the **multi-armed bandit** trade-off:contentReference[oaicite:3]{index=3}.


## 2. Setup & Staging
We configure global constants and disk **staging** helpers so every stage saves a `manifest.json` (plus artifacts). This lets us **resume** safely after any interruption.

In [18]:
# 2) Setup & Staging
import os, json, time, numpy as np, pandas as pd
from pathlib import Path
from datetime import datetime

PROJECT_NAME = "Label_Trainer"
PROJECT_VERSION = "1.4"
RANDOM_STATE = 42
N_THREADS = max(1, os.cpu_count() or 1)

def stage_dir(name: str) -> Path:
    d = Path("staging") / name
    d.mkdir(parents=True, exist_ok=True)
    return d

def save_stage(name: str, manifest: dict, **artifacts):
    d = stage_dir(name)
    (d / "manifest.json").write_text(json.dumps(manifest, indent=2))
    for k, v in artifacts.items():
        if hasattr(v, "to_csv"):
            v.to_csv(d / f"{k}.csv", index=False)
        else:
            import joblib; joblib.dump(v, d / f"{k}.joblib")

def load_stage(name: str):
    d = Path("staging") / name
    m = d / "manifest.json"
    return json.loads(m.read_text()) if m.exists() else None

np.set_printoptions(suppress=True, linewidth=160)
pd.set_option("display.width", 160)


## 3. Data
Load UNSW payloads and map the label to a binary target \($y \in \{0,1\}$\) (1 = attack). Keep **X** as features.

In [19]:
# 3) Data — robust binary label construction (handles strings like 'analysis')

import pandas as pd
import numpy as np

df = pd.read_csv("archive/Payload_data_UNSW.csv")

# Candidates in priority order
CANDIDATES = ["label", "label_str", "attack_cat", "class", "target", "y"]
present = [c for c in CANDIDATES if c in df.columns]
assert present, f"None of the expected label columns found. Present columns: {list(df.columns)[:20]}..."

label_src = present[0]  # pick first available
series = df[label_src]

def is_binary_numeric(s: pd.Series) -> bool:
    # numeric and only {0,1} after dropping NaN
    if not pd.api.types.is_numeric_dtype(s):
        return False
    vals = pd.unique(s.dropna())
    return set(np.unique(vals)).issubset({0, 1})

# Known benign tokens (lowercased)
BENIGN_TOKENS = {"benign", "normal", "background", "bg", "clean"}
# Sometimes datasets have a "Label" column with values {"Attack","Benign"} or multiple attack categories.

if is_binary_numeric(series):
    y = series.astype(int).to_numpy()
    src_used = label_src
else:
    # Treat any non-benign value as attack (1); benign tokens -> 0
    s = series.astype("string").str.strip().str.lower()
    y = (~s.isin(BENIGN_TOKENS)).astype(int).to_numpy()
    src_used = f"{label_src} (string→binary via benign≡0, else≡1)"

# Sanity-check: ensure y ∈ {0,1}
u = np.unique(y[~pd.isna(y)])
assert set(u).issubset({0, 1}), f"Label mapping failed; got values {u}. Inspect column '{label_src}' unique values: {series.unique()[:20]}"

# If we constructed from something other than 'label', ensure we keep an explicit numeric 'label' for consistency
df["label"] = y

# Features X = all except 'label' (keep original text label columns around if you like; here we drop them for modeling)
X = df.drop(columns=["label"])
y = df["label"].values.astype(int)

print(f"Data shape: {df.shape} | Features: {X.shape[1]}")
print(f"Label source used: {src_used}")
print("First 10 mapped labels:", y[:10].tolist())


Data shape: (79881, 1505) | Features: 1504
Label source used: label (string→binary via benign≡0, else≡1)
First 10 mapped labels: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## 4. Train/Test Split & Class Balance
Use **stratified** split to preserve class ratio. Report positive rate, train/test counts. This checks leakage-free generalization setup.

In [20]:
# 4) Split & Class Balance
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=RANDOM_STATE, stratify=y
)
print("Train:", X_train.shape, "| Test:", X_test.shape)
print(f"Train positive rate: {pd.Series(y_train).mean():.4f}")
print("Train dist:", pd.Series(y_train).value_counts().to_dict())
print("Test  dist:", pd.Series(y_test).value_counts().to_dict())


Train: (55916, 1504) | Test: (23965, 1504)
Train positive rate: 0.7371
Train dist: {1: 41216, 0: 14700}
Test  dist: {1: 17665, 0: 6300}


## 5. Preprocess (categoricals/booleans)
LightGBM needs numeric **or** pandas `category` dtypes. We:
- convert `object → category` with aligned levels across train/test,
- convert `bool → int8`.

In [21]:
# Convert object→category with union of categories across train/test
obj_cols_raw = [c for c in X_train.columns if X_train[c].dtype == "object"]
for c in obj_cols_raw:
    tr = X_train[c].astype("string").fillna("__NA__")
    te = X_test[c].astype("string").fillna("__NA__")
    cats = pd.Index(pd.concat([tr, te], axis=0).unique())
    X_train[c] = pd.Categorical(tr, categories=cats)
    X_test[c]  = pd.Categorical(te, categories=cats)

# Booleans → ints
for c in X_train.columns:
    if X_train[c].dtype == bool:
        X_train[c] = X_train[c].astype(np.int8)
        X_test[c]  = X_test[c].astype(np.int8)

# Record categorical columns
cat_cols = [c for c in X_train.columns if str(X_train[c].dtype) == "category"]
print("Categorical columns:", cat_cols)


Categorical columns: ['protocol']


## 6. Metric Helpers
Sweep thresholds \( $\tau$ \) for **max-F1**; compute **TPR/TNR** at a chosen \( $\tau$ \). We optimise **AUPRC** (PR-AUC) and *choose* \( $\tau$ \) by F1 for operations.

In [22]:
from sklearn.metrics import (
    average_precision_score, f1_score, precision_recall_curve, confusion_matrix
)

def sweep_f1(y_true, p_hat, grid=np.linspace(0.05, 0.95, 19)):
    f1s = []
    for t in grid:
        y_hat = (p_hat >= t).astype(int)
        f1s.append(f1_score(y_true, y_hat, zero_division=0))
    best = int(np.argmax(f1s))
    return float(f1s[best]), float(grid[best])

def tpr_tnr_at_tau(y_true, p_hat, tau: float):
    y_hat = (p_hat >= tau).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
    tpr = tp / (tp + fn) if (tp + fn) else 0.0  # recall
    tnr = tn / (tn + fp) if (tn + fp) else 0.0  # specificity
    return float(tpr), float(tnr)


## 7. Baseline (LightGBM)
Sanity-check the pipeline before expensive searches.

In [23]:
# minimal check
import numpy as np
y_true = np.array([0,1,0,1,1,0,0,1])
p_hat  = np.array([0.1,0.8,0.2,0.7,0.9,0.3,0.4,0.6])
best_f1, tau = sweep_f1(y_true, p_hat)
tpr, tnr = tpr_tnr_at_tau(y_true, p_hat, tau)
print(best_f1, tau, tpr, tnr)


1.0 0.44999999999999996 1.0 1.0


In [24]:
# Metric helpers (patch) — safe to re-run anytime
import numpy as np
from sklearn.metrics import average_precision_score, f1_score, confusion_matrix

def sweep_f1(y_true, p_hat, grid=np.linspace(0.05, 0.95, 19)):
    """
    Return (best_f1, tau) by sweeping threshold tau over 'grid'.
    Works for y_true in {0,1} and p_hat as probabilities in [0,1].
    """
    f1s = []
    for t in grid:
        y_hat = (p_hat >= t).astype(int)
        f1s.append(f1_score(y_true, y_hat, zero_division=0))
    best = int(np.argmax(f1s)) if len(f1s) else 0
    return float(f1s[best]), float(grid[best])

def tpr_tnr_at_tau(y_true, p_hat, tau: float):
    """
    Compute TPR (recall) and TNR (specificity) at decision threshold tau.
    """
    y_hat = (p_hat >= tau).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
    tpr = tp / (tp + fn) if (tp + fn) else 0.0
    tnr = tn / (tn + fp) if (tn + fp) else 0.0
    return float(tpr), float(tnr)


In [25]:
from lightgbm import LGBMClassifier, log_evaluation

baseline = LGBMClassifier(
    objective="binary",
    n_estimators=250,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    class_weight="balanced",
    verbosity=-1
)

baseline.fit(
    X_train, y_train,
    categorical_feature=cat_cols or None,
    eval_set=[(X_test, y_test)],
    callbacks=[log_evaluation(period=0)]
)

p_hat_base = baseline.predict_proba(X_test)[:, 1]
auprc_base = average_precision_score(y_test, p_hat_base)
f1_base, tau_base = sweep_f1(y_test, p_hat_base)
tpr_base, tnr_base = tpr_tnr_at_tau(y_test, p_hat_base, tau_base)

print(f"Baseline AUPRC={auprc_base:.4f} | F1@τ={f1_base:.4f} | τ={tau_base:.2f} | TPR={tpr_base:.3f} | TNR={tnr_base:.3f}")


Baseline AUPRC=0.9998 | F1@τ=0.9997 | τ=0.05 | TPR=1.000 | TNR=0.999


## 8. Manual Grid (Parallel + Resume)
Search `(num_leaves, learning_rate)` with `joblib.Parallel`, **checkpoint** results to CSV, and **skip** completed configs on rerun.

In [26]:
from joblib import Parallel, delayed

grid = {
    "num_leaves": [31, 63, 127],
    "learning_rate": [0.05, 0.1, 0.2],
}

def eval_cfg(nl, lr):
    from lightgbm import LGBMClassifier, log_evaluation
    clf = LGBMClassifier(
        objective="binary",
        n_estimators=500,
        num_leaves=nl,
        learning_rate=lr,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        class_weight="balanced",
        verbosity=-1
    )
    clf.fit(X_train, y_train, categorical_feature=cat_cols or None, callbacks=[log_evaluation(period=0)])
    p_hat = clf.predict_proba(X_test)[:, 1]
    auprc = average_precision_score(y_test, p_hat)
    f1, tau = sweep_f1(y_test, p_hat)
    return {"num_leaves": nl, "learning_rate": lr, "AUPRC": auprc, "F1": f1, "tau": tau}

mg_dir = stage_dir("manual_grid")
res_path = mg_dir / "results.csv"
prev = pd.read_csv(res_path) if res_path.exists() else pd.DataFrame()
done = set(zip(prev.get("num_leaves", []), prev.get("learning_rate", [])))

candidates = [(nl, lr) for nl in grid["num_leaves"] for lr in grid["learning_rate"] if (nl, lr) not in done]
if candidates:
    rows = Parallel(n_jobs=min(N_THREADS, len(candidates)))(
        delayed(eval_cfg)(nl, lr) for nl, lr in candidates
    )
    results = pd.concat([prev, pd.DataFrame(rows)], ignore_index=True)
else:
    results = prev

if not results.empty:
    results = results.sort_values(["AUPRC", "F1"], ascending=[False, False]).reset_index(drop=True)
    results.to_csv(res_path, index=False)
    best = results.iloc[0].to_dict()
    # retrain best to persist model & compute TPR/TNR at tau
    from lightgbm import LGBMClassifier, log_evaluation
    best_clf = LGBMClassifier(
        objective="binary", n_estimators=500,
        num_leaves=int(best["num_leaves"]), learning_rate=float(best["learning_rate"]),
        subsample=0.8, colsample_bytree=0.8, random_state=RANDOM_STATE, n_jobs=-1,
        class_weight="balanced", verbosity=-1
    ).fit(X_train, y_train, categorical_feature=cat_cols or None, callbacks=[log_evaluation(period=0)])
    p_best = best_clf.predict_proba(X_test)[:, 1]
    tpr, tnr = tpr_tnr_at_tau(y_test, p_best, float(best["tau"]))
    save_stage("manual_grid", {
        "timestamp": datetime.utcnow().isoformat()+"Z",
        "stage": "manual_grid",
        "model_name": "LightGBM",
        "dataset": "archive/Payload_data_UNSW.csv",
        "seed": RANDOM_STATE,
        "metrics": {"AUPRC": float(best["AUPRC"]), "F1": float(best["F1"]), "tau": float(best["tau"]),
                    "TPR": tpr, "TNR": tnr},
        "params": {"num_leaves": int(best["num_leaves"]), "learning_rate": float(best["learning_rate"])}
    }, results=results, model=best_clf)

display(results.head(10))


/var/folders/sh/njlwqz496112jgym_0_p82h80000gn/T/ipykernel_50502/3808631.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat()+"Z",


,num_leaves,learning_rate,AUPRC,F1,tau
0,31,0.05,0.999763,0.999703,0.05
1,127,0.05,0.999758,0.999660,0.05
2,63,0.05,0.999743,0.999660,0.05
3,63,0.10,0.999712,0.999703,0.05
4,31,0.10,0.999702,0.999703,0.05
5,127,0.20,0.999678,0.999660,0.05
6,127,0.10,0.999666,0.999660,0.05
7,63,0.20,0.999602,0.999660,0.05
8,31,0.20,0.999592,0.999703,0.05


## 8b. Automated HPO — Randomized Search (Stratified CV, resumable)
We use `RandomizedSearchCV` to explore a larger space quickly. Objective = **PR-AUC** (Average Precision).
- **Resume-safe:** results saved to `staging/random_search/` (CSV + manifest).
- **Fit-time cat handling:** pass `categorical_feature=cat_cols` to LightGBM during `fit`.

In [ ]:
# 8b) Automated HPO — Randomized Search (resumable)

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import randint, loguniform
from lightgbm import LGBMClassifier

rs_dir = stage_dir("random_search")
rs_path = rs_dir / "results.csv"

param_distributions = {
    "num_leaves": randint(16, 256),
    "max_depth": randint(2, 16),
    "min_child_samples": randint(10, 200),
    "subsample": loguniform(0.6, 1.0),          # sampled in (0.6, 1.0]
    "colsample_bytree": loguniform(0.6, 1.0),
    "learning_rate": loguniform(1e-3, 2e-1),
    "reg_lambda": loguniform(1e-3, 10.0),
}

rs_base = LGBMClassifier(
    objective="binary",
    n_estimators=500,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    class_weight="balanced",
    verbosity=-1
)

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

rs = RandomizedSearchCV(
    estimator=rs_base,
    param_distributions=param_distributions,
    n_iter=40,
    scoring="average_precision",
    cv=cv,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1,
    refit=True  # refit best on full training folds
)

# pass categorical_feature to the underlying estimator during fit
fit_kwargs = {"categorical_feature": cat_cols or None}
rs.fit(X_train, y_train, **fit_kwargs)

best_rs = rs.best_estimator_
p_rs = best_rs.predict_proba(X_test)[:, 1]
auprc_rs = average_precision_score(y_test, p_rs)
f1_rs, tau_rs = sweep_f1(y_test, p_rs)
tpr_rs, tnr_rs = tpr_tnr_at_tau(y_test, p_rs, tau_rs)

# Persist result table (cv_results_) and manifest
res_df = pd.DataFrame(rs.cv_results_)
res_df.to_csv(rs_path, index=False)

save_stage("random_search", {
    "timestamp": datetime.utcnow().isoformat() + "Z",
    "stage": "random_search",
    "model_name": "LightGBM_RS",
    "dataset": "archive/Payload_data_UNSW.csv",
    "seed": RANDOM_STATE,
    "metrics": {"AUPRC": float(auprc_rs), "F1": float(f1_rs), "tau": float(tau_rs),
                "TPR": float(tpr_rs), "TNR": float(tnr_rs)},
    "best_params": rs.best_params_
}, results=res_df, model=best_rs)

print("Randomized Search | AUPRC={:.4f} F1@τ={:.4f} τ={:.2f}".format(auprc_rs, f1_rs, tau_rs))


Fitting 3 folds for each of 40 candidates, totalling 120 fits


## 9. Bayesian Optimisation (Checkpoint & Resume)
`BayesSearchCV` + `CheckpointSaver`. Objective = **PR-AUC**. After BO, select **τ** by max-F1 on test.

In [ ]:
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from skopt.callbacks import CheckpointSaver
from sklearn.model_selection import StratifiedKFold

bo_dir = stage_dir('bo_lgb')
ckpt = bo_dir / 'skopt_ckpt.pkl'

lgb = LGBMClassifier(
    objective='binary',
    n_estimators=500,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    class_weight="balanced",
    verbosity=-1
)

search_spaces = {
    'num_leaves': Integer(31, 255),
    'max_depth': Integer(2, 16),
    'min_child_samples': Integer(10, 200),
    'subsample': Real(0.6, 1.0),
    'colsample_bytree': Real(0.6, 1.0),
    'learning_rate': Real(1e-3, 2e-1, prior='log-uniform'),
    'reg_lambda': Real(1e-3, 10.0, prior='log-uniform')
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
callbacks = [CheckpointSaver(str(ckpt), compress=9)]  # correct signature

opt = BayesSearchCV(
    estimator=lgb,
    search_spaces=search_spaces,
    n_iter=40,
    scoring='average_precision',
    cv=cv,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1
)

opt.fit(X_train, y_train, callback=callbacks)

print('Best params:', opt.best_params_)
print(f'Best CV PR-AUC: {opt.best_score_:.4f}')
lgb_bo = opt.best_estimator_

y_pred_proba_bo = lgb_bo.predict_proba(X_test)[:, 1]
auprc_bo = average_precision_score(y_test, y_pred_proba_bo)
f1_bo, tau_bo = sweep_f1(y_test, y_pred_proba_bo)
tpr_bo, tnr_bo = tpr_tnr_at_tau(y_test, y_pred_proba_bo, tau_bo)

save_stage('bo_lgb', {
    'timestamp': time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime()),
    'stage': 'bo_lgb',
    'model_name': 'LightGBM_BO',
    'dataset': 'archive/Payload_data_UNSW.csv',
    'seed': RANDOM_STATE,
    'metrics': {'AUPRC': float(auprc_bo), 'F1': float(f1_bo), 'tau': float(tau_bo),
                'TPR': float(tpr_bo), 'TNR': float(tnr_bo)},
    'best_params': opt.best_params_
}, model=lgb_bo)


## 9b. Enhanced BO — ask–tell (warm start, batch EI, patience, resume)
We switch to `skopt.Optimizer` for fine control:
- **Warm start** from Manual Grid + Randomized Search winners
- **Batch suggestions** (`n_points`) for multi-core eval
- **Patience** (early-stop if no PR-AUC improvement)
- **Resume** from `staging/bo_enhanced/optimizer.pkl`

In [ ]:
# 9b) Enhanced BO — ask–tell (warm start, batch, patience, resume)

from skopt import Optimizer
from skopt.space import Integer, Real
from joblib import Parallel, delayed

ben_dir = stage_dir("bo_enhanced")
opt_ckpt = ben_dir / "optimizer.pkl"
res_csv  = ben_dir / "results.csv"

# Search space
space = [
    Integer(31, 255, name="num_leaves"),
    Integer(2, 16, name="max_depth"),
    Integer(10, 200, name="min_child_samples"),
    Real(0.6, 1.0, name="subsample"),
    Real(0.6, 1.0, name="colsample_bytree"),
    Real(1e-3, 2e-1, prior="log-uniform", name="learning_rate"),
    Real(1e-3, 10.0, prior="log-uniform", name="reg_lambda"),
]

# Build warm-start points from staged results if available
X0, y0 = [], []
for stage in ["manual_grid", "random_search", "bo_lgb"]:
    m = load_stage(stage)
    if m and "params" in m:
        p = m["params"]
        if stage == "bo_lgb" and "best_params" in m:
            p = m["best_params"]
        x = [int(p.get("num_leaves", 63)),
             int(p.get("max_depth", 6)),
             int(p.get("min_child_samples", 20)),
             float(p.get("subsample", 0.8)),
             float(p.get("colsample_bytree", 0.8)),
             float(p.get("learning_rate", 0.1)),
             float(p.get("reg_lambda", 1.0))]
        # If metrics present, use negative AUPRC (Optimizer minimizes)
        ap = m["metrics"].get("AUPRC") if "metrics" in m else None
        if ap is not None:
            X0.append(x); y0.append(-float(ap))

# Initialize or resume Optimizer
if opt_ckpt.exists():
    import joblib
    opt = joblib.load(opt_ckpt)
else:
    opt = Optimizer(dimensions=space, base_estimator="GP", acq_func="EI", random_state=RANDOM_STATE)
    if X0:
        opt.tell(X0, y0)  # warm-start

# CV evaluator (3-fold AP) with LightGBM + cat cols
from sklearn.model_selection import StratifiedKFold

def ap_cv(params_dict):
    clf = LGBMClassifier(
        objective="binary", n_estimators=500, random_state=RANDOM_STATE,
        n_jobs=-1, class_weight="balanced", verbosity=-1, **params_dict
    )
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    ap_scores = []
    for tr_idx, va_idx in cv.split(X_train, y_train):
        X_tr, X_va = X_train.iloc[tr_idx], X_train.iloc[va_idx]
        y_tr, y_va = y_train[tr_idx], y_train[va_idx]
        clf.fit(X_tr, y_tr, categorical_feature=cat_cols or None)
        p = clf.predict_proba(X_va)[:, 1]
        ap_scores.append(average_precision_score(y_va, p))
    return float(np.mean(ap_scores))

# Ask–tell loop (batch), with patience & checkpoint
N_ITERS   = 30
BATCH     = min(4, N_THREADS)  # parallel batch size
PATIENCE  = 6
best_ap   = max([-y for y in y0], default=-np.inf)
no_improve= 0

# Load prior CSV if exists
hist = pd.read_csv(res_csv) if res_csv.exists() else pd.DataFrame(columns=[
    "num_leaves","max_depth","min_child_samples","subsample","colsample_bytree","learning_rate","reg_lambda","AP_cv"
])

for step in range(N_ITERS):
    X_batch = opt.ask(n_points=BATCH)
    # Convert list→dict for evaluator
    def vec_to_params(v):
        return {
            "num_leaves": int(v[0]), "max_depth": int(v[1]), "min_child_samples": int(v[2]),
            "subsample": float(v[3]), "colsample_bytree": float(v[4]),
            "learning_rate": float(v[5]), "reg_lambda": float(v[6])
        }

    params_batch = [vec_to_params(v) for v in X_batch]
    scores = Parallel(n_jobs=BATCH)(
        delayed(ap_cv)(p) for p in params_batch
    )
    # Optimizer minimizes, so pass negative AP
    opt.tell(X_batch, [-s for s in scores])

    # Update history
    rows = [dict(**p, AP_cv=s) for p, s in zip(params_batch, scores)]
    hist = pd.concat([hist, pd.DataFrame(rows)], ignore_index=True)
    hist.to_csv(res_csv, index=False)

    # Early stopping on AP
    batch_best = max(scores)
    if batch_best > best_ap + 1e-6:
        best_ap = batch_best
        no_improve = 0
    else:
        no_improve += 1

    # Checkpoint optimizer
    import joblib
    joblib.dump(opt, opt_ckpt)

    print(f"[Enhanced BO] step {step+1}/{N_ITERS} | best_cv_AP={best_ap:.4f} | patience={no_improve}/{PATIENCE}")
    if no_improve >= PATIENCE:
        print("Early stopping (patience reached).")
        break

# Select best params from history, retrain on full train, evaluate on test
if not hist.empty:
    best_row = hist.sort_values("AP_cv", ascending=False).iloc[0].to_dict()
    best_params_enh = {k: best_row[k] for k in ["num_leaves","max_depth","min_child_samples","subsample","colsample_bytree","learning_rate","reg_lambda"]}
    best_enh = LGBMClassifier(
        objective="binary", n_estimators=500, random_state=RANDOM_STATE, n_jobs=-1,
        class_weight="balanced", verbosity=-1, **best_params_enh
    ).fit(X_train, y_train, categorical_feature=cat_cols or None)

    p_enh = best_enh.predict_proba(X_test)[:, 1]
    auprc_enh = average_precision_score(y_test, p_enh)
    f1_enh, tau_enh = sweep_f1(y_test, p_enh)
    tpr_enh, tnr_enh = tpr_tnr_at_tau(y_test, p_enh, tau_enh)

    save_stage("bo_enhanced", {
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "stage": "bo_enhanced",
        "model_name": "LightGBM_BO_Enhanced",
        "dataset": "archive/Payload_data_UNSW.csv",
        "seed": RANDOM_STATE,
        "metrics": {"AUPRC": float(auprc_enh), "F1": float(f1_enh), "tau": float(tau_enh),
                    "TPR": float(tpr_enh), "TNR": float(tnr_enh)},
        "params": best_params_enh
    }, results=hist, model=best_enh)

    print("Enhanced BO | AUPRC={:.4f} F1@τ={:.4f} τ={:.2f}".format(auprc_enh, f1_enh, tau_enh))
else:
    print("Enhanced BO produced no evaluations.")


## 10. Champion Selection
Compare **Manual Grid** vs **BO** by AUPRC then F1. Declare the **champion**.

In [ ]:
# 10) Champion Selection — now includes Random Search and Enhanced BO

metrics = {}
for stage in ["manual_grid", "random_search", "bo_lgb", "bo_enhanced"]:
    m = load_stage(stage)
    if m and "metrics" in m:
        metrics[stage] = m["metrics"]

if metrics:
    dfm = pd.DataFrame(metrics).T
    display(dfm)
    champ_name = dfm.sort_values(["AUPRC", "F1"], ascending=[False, False]).index[0]
    champ = dfm.loc[champ_name].to_dict()
    print("Champion:", champ_name, "|", champ)
else:
    champ_name, champ = None, None
    print("No metrics found.")


## 11. SOC Report (TPR/TNR @ τ) + Confusion Matrix
Operational view for detection coverage (**TPR**) vs false-positives (**TNR**) at the selected \( \tau \).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

def soc_report(y_true, p_hat, tau):
    y_hat = (p_hat >= tau).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
    tpr = tp / (tp + fn) if (tp + fn) else 0.0
    tnr = tn / (tn + fp) if (tn + fp) else 0.0
    print(f"τ={tau:.2f} | TPR={tpr:.3f} | TNR={tnr:.3f} | TP={tp} FP={fp} TN={tn} FN={fn}")
    ConfusionMatrixDisplay.from_predictions(y_true, y_hat)
    plt.title(f"Confusion Matrix @ τ={tau:.2f}")
    plt.show()

# Prefer BO if available
if bo:
    soc_report(y_test, y_pred_proba_bo, tau_bo)
elif mg:
    # fallback to baseline if needed
    soc_report(y_test, p_hat_base, tau_base)
else:
    soc_report(y_test, p_hat_base, tau_base)


## 12. Auto-Wire Docs
Inject latest **AUPRC / F1 / τ** into `model_card.md` and `README.md` between `<!--METRICS_START--> … <!--METRICS_END-->` (append if absent).

In [ ]:
from pathlib import Path

manifest = load_stage("bo_lgb") or load_stage("manual_grid")
if manifest:
    m = manifest["metrics"]
    auprc = m.get("AUPRC") or m.get("auprc")
    f1 = m.get("F1") or m.get("f1")
    tau = m.get("tau")
    block = (
        "<!--METRICS_START-->\n"
        f"AUPRC: {auprc:.4f}\n"
        f"F1: {f1:.4f}\n"
        f"τ: {tau:.2f}\n"
        "<!--METRICS_END-->\n"
    )
    for doc in ["model_card.md", "README.md"]:
        p = Path(doc)
        if not p.exists(): 
            continue
        txt = p.read_text()
        if "<!--METRICS_START-->" in txt and "<!--METRICS_END-->" in txt:
            pre, _, rest = txt.partition("<!--METRICS_START-->")
            _, _, post = rest.partition("<!--METRICS_END-->")
            txt = pre + block + post
        else:
            if not txt.endswith("\n"):
                txt += "\n"
            txt += "\n" + block
        p.write_text(txt)
    print("Docs updated.")
else:
    print("No manifest available to auto-wire docs.")


## 13. Version Log
Append a versioned entry to the changelog for auditability.

In [ ]:
log = Path("CHANGELOG_Label_Trainer.md")
entry = f"- {datetime.utcnow().isoformat()}Z | v{PROJECT_VERSION} | v1.4 pipeline: staging/resume, manual grid, BO checkpoint, SOC report, auto-wire\n"
if log.exists():
    prev = log.read_text()
    if entry not in prev:
        log.write_text(prev + entry)
else:
    log.write_text(entry)
print("CHANGELOG updated.")


## Tools for justify BO & hyperparams

* Tables (Cell 14): compact, reproducible evidence across all methods.

* Convergence (Cell 15): shows best-so-far CV AP improving as BO progresses.

* Response plots (Cell 16): illustrates sensitivity of AP vs hyperparameters and supports the selected values.

* PR curves (Cell 17): operationally relevant under class imbalance (AUPRC focus).

* F1 vs τ (Cell 18): makes threshold choice transparent; useful for SOC policy.

* Feature importance (Cell 19): supports LightGBM interpretability claims.

* CNN benchmark (Cell 20): fair comparison if you have a 1D-CNN baseline; otherwise it’s a clean slot to drop results into.

* Champion auto-wire (Cell 21): pushes the winner into your docs.

## 14. Aggregate Metrics Table (Manual, Random, BO, Enhanced BO, CNN)
Summarise **AUPRC**, **F1**, **τ**, **TPR**, **TNR** for each stage. If a 1D-CNN benchmark is staged under `staging/cnn1d/manifest.json`, it is included automatically.


In [ ]:
# 14) Aggregate metrics table
import json
from pathlib import Path

stages = ["manual_grid", "random_search", "bo_lgb", "bo_enhanced", "cnn1d"]
rows = []
for s in stages:
    m = load_stage(s)
    if not m or "metrics" not in m:
        continue
    r = {"stage": s}
    r.update({k: m["metrics"].get(k) for k in ["AUPRC", "F1", "tau", "TPR", "TNR"]})
    rows.append(r)

metrics_df = pd.DataFrame(rows)
if not metrics_df.empty:
    display(metrics_df.sort_values(["AUPRC","F1"], ascending=[False, False]).reset_index(drop=True))
else:
    print("No staged metrics found. Run training and HPO first.")


## 15. BO Convergence
Show optimisation progress:
- **BayesSearchCV**: proxy best-so-far from CV (if history available).
- **Enhanced BO**: `AP_cv` per step from `staging/bo_enhanced/results.csv`.
This justifies BO’s **sample efficiency** vs manual/random search.


In [ ]:
# 15) BO convergence plots (matplotlib only)
import matplotlib.pyplot as plt

# Enhanced BO history
ben_dir = Path("staging/bo_enhanced")
ben_hist = ben_dir / "results.csv"

plt.figure()
if ben_hist.exists():
    h = pd.read_csv(ben_hist)
    if not h.empty and "AP_cv" in h.columns:
        best_so_far = h["AP_cv"].cummax().values
        plt.plot(range(1, len(best_so_far)+1), best_so_far, marker='o', linewidth=1)
        plt.xlabel("Enhanced BO evaluations")
        plt.ylabel("Best CV AP so far")
        plt.title("Enhanced BO Convergence (best-so-far CV AP)")
        plt.grid(True, linewidth=0.3)
    else:
        plt.text(0.5, 0.5, "No AP_cv history yet.", ha='center')
else:
    plt.text(0.5, 0.5, "staging/bo_enhanced/results.csv not found.", ha='center')
plt.show()

# BayesSearchCV does not expose per-iteration history portably; show final best if staged
bo = load_stage("bo_lgb")
if bo and "metrics" in bo:
    print("BayesSearchCV best (test): AUPRC={:.4f}, F1@τ={:.4f}, τ={:.2f}".format(
        bo["metrics"].get("AUPRC", float('nan')),
        bo["metrics"].get("F1", float('nan')),
        bo["metrics"].get("tau", float('nan')),
    ))


## 16. Hyperparameter Response (From Enhanced BO History)
Visual diagnostics of **hyperparameters vs CV AP** (scatter) + a **parallel coordinates-style** view (numeric columns). These help justify the **chosen hyperparameters** and show sensitivity.


In [ ]:
# 16) Hyperparameter response plots from enhanced BO history
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates

ben_hist = Path("staging/bo_enhanced/results.csv")
if ben_hist.exists():
    h = pd.read_csv(ben_hist)
    if not h.empty and "AP_cv" in h.columns:
        # Scatter: each hyperparam vs AP
        hp_cols = ["num_leaves","max_depth","min_child_samples","subsample","colsample_bytree","learning_rate","reg_lambda"]
        for c in hp_cols:
            if c in h.columns:
                plt.figure()
                plt.scatter(h[c].values, h["AP_cv"].values, s=12)
                plt.xlabel(c)
                plt.ylabel("CV AP")
                plt.title(f"Hyperparameter response: {c} vs CV AP")
                plt.grid(True, linewidth=0.3)
                plt.show()

        # Parallel-coordinates-style view (bucket top/bottom)
        try:
            tmp = h.copy()
            # Bucket AP into categories to plot
            q = tmp["AP_cv"].quantile([0.25, 0.5, 0.75]).to_dict()
            def bucket(x):
                if x >= q[0.75]: return "top"
                if x <= q[0.25]: return "bottom"
                return "mid"
            tmp["bucket"] = tmp["AP_cv"].apply(bucket)
            pc_cols = [c for c in hp_cols if c in tmp.columns]
            if pc_cols:
                pc_df = tmp[pc_cols + ["bucket"]].copy()
                plt.figure(figsize=(10, 5))
                # fallback simple parallel plot: normalize columns
                norm = pc_df[pc_cols].astype(float)
                norm = (norm - norm.min()) / (norm.max() - norm.min() + 1e-12)
                norm["bucket"] = pc_df["bucket"]
                # emulate parallel_coordinates (without color control)
                for label in ["bottom","mid","top"]:
                    sub = norm[norm["bucket"] == label]
                    if sub.empty: 
                        continue
                    for _, row in sub.iterrows():
                        plt.plot(range(len(pc_cols)), row[pc_cols].values, alpha=0.3)
                plt.xticks(range(len(pc_cols)), pc_cols, rotation=45, ha="right")
                plt.ylabel("Normalized hyperparameter value")
                plt.title("Parallel view of top/mid/bottom AP configurations")
                plt.grid(True, linewidth=0.3)
                plt.tight_layout()
                plt.show()
        except Exception as e:
            print("Parallel view failed:", e)
    else:
        print("Enhanced BO history has no AP_cv column.")
else:
    print("Enhanced BO history not found.")


## 17. Precision–Recall Curves (Champion vs Baseline vs 1D-CNN)
Compare **PR curves** to justify BO’s gains where it matters (high precision at actionable recall).


In [ ]:
# 17) PR Curves for comparison: champion vs baseline vs CNN (if available)
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score

# Helper to compute PR arrays from proba
def pr_arrays(y_true, p_hat):
    p, r, t = precision_recall_curve(y_true, p_hat)
    ap = average_precision_score(y_true, p_hat)
    return p, r, ap

# Champion = prefer enhanced BO, then BO, then random, then manual, else baseline
p_preds = {}
labels = {}

# baseline
p_preds["baseline"] = p_hat_base
labels["baseline"] = ("Baseline", average_precision_score(y_test, p_hat_base))

# staged champions
order = ["bo_enhanced","bo_lgb","random_search","manual_grid"]
for s in order:
    m = load_stage(s)
    if not m: 
        continue
    # Attempt to re-create proba if model saved; else skip curve but show AP from manifest
    model_path = Path("staging")/s/"model.joblib"
    if model_path.exists():
        import joblib
        mdl = joblib.load(model_path)
        p_preds[s] = mdl.predict_proba(X_test)[:,1]
        labels[s] = (s, average_precision_score(y_test, p_preds[s]))

# CNN (if staged)
cnn = load_stage("cnn1d")
if cnn:
    cnn_model = Path("staging")/"cnn1d"/"model.joblib"
    if cnn_model.exists():
        import joblib
        mdl = joblib.load(cnn_model)
        # Expect a predict_proba-like method or predict giving probabilities
        try:
            p_preds["cnn1d"] = mdl.predict_proba(X_test)[:, 1]
        except Exception:
            # fallback: if predict returns logits or probs, adapt as needed
            p_preds["cnn1d"] = np.clip(mdl.predict(X_test), 0, 1).astype(float).ravel()
        labels["cnn1d"] = ("CNN-1D", average_precision_score(y_test, p_preds["cnn1d"]))

# Plot
plt.figure()
for k, p in p_preds.items():
    P, R, AP = pr_arrays(y_test, p)
    plt.plot(R, P, linewidth=1, label=f"{k} (AP={AP:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision–Recall Curves")
plt.grid(True, linewidth=0.3)
plt.legend()
plt.show()


## 18. Threshold vs F1 (Champion vs 1D-CNN)
Show \( $F1(\tau)$ \) to justify the **operational threshold** choice for BO vs 1D-CNN.


In [ ]:
# 18) Threshold vs F1 curve comparison
import matplotlib.pyplot as plt

taus = np.linspace(0.01, 0.99, 50)
def f1_curve(y_true, p_hat, ts):
    vals = []
    for t in ts:
        vals.append(f1_score(y_true, (p_hat >= t).astype(int), zero_division=0))
    return np.array(vals)

curves = {}
# champion: prefer bo_enhanced then bo_lgb then random_search then manual_grid then baseline
pref = None
for s in ["bo_enhanced","bo_lgb","random_search","manual_grid"]:
    if s in p_preds:
        pref = s
        break
if pref:
    curves[pref] = f1_curve(y_test, p_preds[pref], taus)
else:
    curves["baseline"] = f1_curve(y_test, p_preds["baseline"], taus)

# cnn if available
if "cnn1d" in p_preds:
    curves["cnn1d"] = f1_curve(y_test, p_preds["cnn1d"], taus)

plt.figure()
for name, vals in curves.items():
    plt.plot(taus, vals, linewidth=1, label=name)
plt.xlabel("τ")
plt.ylabel("F1")
plt.title("F1 vs τ (threshold sweep)")
plt.grid(True, linewidth=0.3)
plt.legend()
plt.show()


## 19. Feature Importance (Champion LightGBM)
Bar chart of **gain** importances from the champion LightGBM to justify interpretability and selected hyperparameters.


In [ ]:
# 19) Feature importance for champion LightGBM (if available)
import matplotlib.pyplot as plt

champ_model = None
for s in ["bo_enhanced","bo_lgb","random_search","manual_grid"]:
    p = Path("staging")/s/"model.joblib"
    if p.exists():
        import joblib
        champ_model = joblib.load(p)
        champ_stage = s
        break

if champ_model is not None and hasattr(champ_model, "booster_"):
    booster = champ_model.booster_
    try:
        imp = booster.feature_importance(importance_type="gain")
        names = booster.feature_name()
        imp_df = pd.DataFrame({"feature": names, "gain": imp}).sort_values("gain", ascending=False).head(20)
        plt.figure(figsize=(8, 6))
        plt.barh(range(len(imp_df)), imp_df["gain"].values)
        plt.yticks(range(len(imp_df)), imp_df["feature"].values)
        plt.gca().invert_yaxis()
        plt.xlabel("Gain Importance")
        plt.title(f"Top-20 Feature Importances ({champ_stage})")
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print("Could not extract LightGBM importances:", e)
else:
    print("Champion LightGBM model not found or missing booster_.")


## 20. 1D-CNN Benchmark Loader
If you trained a 1D-CNN outside this notebook, drop its test predictions or model into `staging/cnn1d/`:
- `model.joblib` with `predict_proba(X_test)[:,1]` or `predict(X_test)` in \([0,1]\),
- or a `preds.npy` of probabilities.
Then run this cell to compute **AUPRC/F1/τ** and stage a manifest for fair comparison.


In [ ]:
# 20) 1D-CNN benchmark loader → stage manifest

cnn_dir = stage_dir("cnn1d")
cnn_model = cnn_dir / "model.joblib"
cnn_preds = cnn_dir / "preds.npy"

p_cnn = None
if cnn_preds.exists():
    p_cnn = np.load(cnn_preds)
elif cnn_model.exists():
    import joblib
    mdl = joblib.load(cnn_model)
    try:
        p_cnn = mdl.predict_proba(X_test)[:, 1]
    except Exception:
        p_cnn = np.clip(mdl.predict(X_test), 0, 1).astype(float).ravel()

if p_cnn is not None:
    auprc_cnn = average_precision_score(y_test, p_cnn)
    f1_cnn, tau_cnn = sweep_f1(y_test, p_cnn)
    tpr_cnn, tnr_cnn = tpr_tnr_at_tau(y_test, p_cnn, tau_cnn)
    save_stage("cnn1d", {
        "timestamp": datetime.utcnow().isoformat()+"Z",
        "stage": "cnn1d",
        "model_name": "CNN1D",
        "dataset": "archive/Payload_data_UNSW.csv",
        "seed": RANDOM_STATE,
        "metrics": {"AUPRC": float(auprc_cnn), "F1": float(f1_cnn), "tau": float(tau_cnn),
                    "TPR": float(tpr_cnn), "TNR": float(tnr_cnn)}
    })
    print("CNN1D staged | AUPRC={:.4f} F1@τ={:.4f} τ={:.2f}".format(auprc_cnn, f1_cnn, tau_cnn))
else:
    print("No CNN predictions/model found in staging/cnn1d/. Provide model.joblib or preds.npy.")


## 21. Champion-Driven Auto-Wire (optional)
Auto-wire **the current champion** (best AUPRC → F1) into `model_card.md` and `README.md`.


In [ ]:
# 21) Champion-driven auto-wire
# Reuse previous champion selection if dfm exists; else recompute here
if 'dfm' not in globals() or dfm.empty:
    metrics = {}
    for s in ["manual_grid", "random_search", "bo_lgb", "bo_enhanced", "cnn1d"]:
        m = load_stage(s)
        if m and "metrics" in m:
            metrics[s] = m["metrics"]
    dfm = pd.DataFrame(metrics).T if metrics else pd.DataFrame()

if not dfm.empty:
    champ_name = dfm.sort_values(["AUPRC", "F1"], ascending=[False, False]).index[0]
    m = load_stage(champ_name)["metrics"]
    auprc = m.get("AUPRC") or m.get("auprc")
    f1 = m.get("F1") or m.get("f1")
    tau = m.get("tau")
    block = (
        "<!--METRICS_START-->\n"
        f"Stage: {champ_name}\n"
        f"AUPRC: {auprc:.4f}\n"
        f"F1: {f1:.4f}\n"
        f"τ: {tau:.2f}\n"
        "<!--METRICS_END-->\n"
    )
    for doc in ["model_card.md", "README.md"]:
        p = Path(doc)
        if not p.exists(): 
            continue
        txt = p.read_text()
        if "<!--METRICS_START-->" in txt and "<!--METRICS_END-->" in txt:
            pre, _, rest = txt.partition("<!--METRICS_START-->")
            _, _, post = rest.partition("<!--METRICS_END-->")
            txt = pre + block + post
        else:
            if not txt.endswith("\n"):
                txt += "\n"
            txt += "\n" + block
        p.write_text(txt)
    print("Docs updated with champion metrics:", champ_name)
else:
    print("No metrics available to auto-wire champion.")
